In [1]:
import fsspec
import os
import json
from tqdm.auto import tqdm
import json


In [6]:
with open('/home/gpubox3/projects/hf-exp/model/classes-rafflesia-dataset-23986-2023-12-03.json', 'r') as f:
    data = json.load(f)

with open('/home/gpubox3/projects/hf-exp/model/classes-rafflesia.txt', 'w') as f:
    f.writelines('\n'.join(data.keys()))

In [7]:
metadata_url = 's3://rafflesia-base-dataset/_metadata'

fs, url_path = fsspec.core.url_to_fs(metadata_url)

with fs.open(os.path.join(url_path, 'classes-rafflesia-dataset-23986-2023-12-03.json'), "r") as f:
    index_to_name = json.load(f)


In [ ]:
# s5cmd mv "s3://rafflesia-base-dataset/data/train/*.json" s3://rafflesia-base-dataset/data/train/_metadata/
# s5cmd mv "s3://rafflesia-base-dataset/data/train/*.parquet" s3://rafflesia-base-dataset/data/train/_metadata/

In [ ]:
# s5cmd mv "s3://rafflesia-base-dataset/data/val//*.parquet" s3://rafflesia-base-dataset/data/val/_metadata/
# s5cmd mv "s3://rafflesia-base-dataset/data/val/*.json" s3://rafflesia-base-dataset/data/val/_metadata/

In [33]:
metadata_train_url = 's3://rafflesia-base-dataset/data/val/_metadata'

fs_train, url_path_train = fsspec.core.url_to_fs(metadata_train_url)

info_json_data = {}

metadata_files = list(sorted(fs_train.glob(url_path_train + "/*.json")))


In [34]:
len(metadata_files)

60

In [35]:
metadata_files[1]

'rafflesia-base-dataset/data/val/_metadata/00000001_stats.json'

In [36]:
omm_shard_count = max([len(os.path.basename(_file).split('_')[0]) for _file in metadata_files])

def _get_shard_name(shard_id, oom_shard_count):
    return "{shard_id:0{oom_shard_count}d}".format(
        shard_id=shard_id, oom_shard_count=oom_shard_count
    )

split_name = f"{{{_get_shard_name(0, omm_shard_count)}..{_get_shard_name(len(metadata_files)-1, omm_shard_count)}}}.tar"

info_structure = {
    "splits": {
        f"{split_name}": {
            "name": "train",
            "filenames": [],
            "shard_lengths": []
        }
    }
}

for _meta_file in tqdm(metadata_files):
    filename = os.path.basename(_meta_file).split('_')[0]

    info_structure['splits'][f"{split_name}"]['filenames'].append(f"{filename}.tar")

    with fs.open(_meta_file, "r") as f:
        _metadata = json.load(f)
        length = _metadata['successes']

    info_structure['splits'][f"{split_name}"]['shard_lengths'].append(int(length))

def save_info_json(info_structure, train_path_url='s3://rafflesia-base-dataset/data/train'):
    fs, url_path_train = fsspec.core.url_to_fs(train_path_url)

    with fs.open(os.path.join(url_path_train, 'info.json'), mode='w') as f:
        json.dump(info_structure, f, indent=4)

# save_info_json(info_structure)
save_info_json(info_structure, 's3://rafflesia-base-dataset/data/val')


  0%|          | 0/60 [00:00<?, ?it/s]